In [1]:
# ==========================================================
# GLOBAL PERFORMANCE CONTROL (SET BEFORE ML IMPORTS)
# ==========================================================
import os
import shutil
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"


In [8]:

INTENT_MODEL_PATH = "models/intent_model_optimized.joblib"

ASR_MODELS = {
    "coqui_stt": "models\ASR_STT\coqui_stt\coqui_stt_asr_model.tflite",
    "wav2vec2": "models\ASR_STT\IndicWav2Vec2\pytorch_model.bin",
    # "wav2vec2": "models\ASR_STT\wav2vec2\model.safetensors",
    "vosk": "models/ASR_STT/vosk-model-small-hi-0.22/vosk-model-small-hi-0.22",
    "whisper": "models\ASR_STT\Wisper-small\model.safetensors"
}

ASR_model_select="coqui_stt"

# System binaries
SYSTEM_TTS_ENGINES = {
    "espeak-ng": "models/espeak-ng",
    "festival": "models/festival"
}

TTS_model_select="espeak-ng"

TRANSLATION_MODEL={
    "hi-en_model": "models\Translation_model\hi-en_model"
}

TRANSLATION_model_select="hi-en_model"

In [9]:
def preload_asr_model():
    print("Loading ASR model (Optimized)...")

    asr_model_path = ASR_MODELS.get(ASR_model_select)

    # ------------------------------------------------------
    # COQUI STT (TFLite - Already Quantized)
    # ------------------------------------------------------
    if ASR_model_select == "coqui_stt":
        try:
            import stt  # import only if selected
            model = stt.Model(asr_model_path)
            print("✔ Coqui STT (TFLite Quantized) Loaded")
            return model
        except Exception as e:
            print("ASR Load Error:", e)
            return None

    # ------------------------------------------------------
    # WAV2VEC2 (Dynamic Quantization)
    # ------------------------------------------------------
    elif ASR_model_select == "wav2vec2":
        try:
            import torch
            from transformers import Wav2Vec2ForCTC

            torch.set_num_threads(2)
            torch.set_grad_enabled(False)

            model = Wav2Vec2ForCTC.from_pretrained(
                asr_model_path,
                local_files_only=True
            )

            model.eval()

            # Dynamic INT8 Quantization
            model = torch.quantization.quantize_dynamic(
                model,
                {torch.nn.Linear},
                dtype=torch.qint8
            )

            print("✔ Wav2Vec2 (Dynamic INT8 Quantized) Loaded")
            return model

        except Exception as e:
            print("ASR Load Error:", e)
            return None

    # ------------------------------------------------------
    # VOSK (Already Lightweight Kaldi Model)
    # ------------------------------------------------------
    elif ASR_model_select == "vosk":
        try:
            from vosk import Model, KaldiRecognizer

            model = Model(asr_model_path)
            recognizer = KaldiRecognizer(model, SAMPLE_RATE)

            print("✔ Vosk Lightweight Model Loaded")
            return recognizer

        except Exception as e:
            print("ASR Load Error:", e)
            return None

    # ------------------------------------------------------
    # WHISPER (CPU Quantized)
    # ------------------------------------------------------
    elif ASR_model_select == "whisper":
        try:
            import whisper
            model = whisper.load_model(
                "small",
                device="cpu"
            )

            print("✔ Whisper CPU Model Loaded")
            return model

        except Exception as e:
            print("ASR Load Error:", e)
            return None

    return None


In [10]:
def preload_intent_model():
    print("Loading Intent Model (Optimized)...")

    try:
        import joblib
        model = joblib.load(INTENT_MODEL_PATH)
        print("✔ Intent Model Loaded")
        return model
    except Exception as e:
        print("Intent Load Error:", e)
        return None


In [11]:
def preload_tts_model():
    print("Loading TTS Engine...")

    if TTS_model_select == "espeak-ng":
        if shutil.which("espeak-ng"):
            print("✔ eSpeak-NG Ready (System Level)")
            return "espeak-ng"
        else:
            print("✖ eSpeak-NG Not Found")
            return None

    elif TTS_model_select == "festival":
        if shutil.which("festival"):
            print("✔ Festival Ready (System Level)")
            return "festival"
        else:
            print("✖ Festival Not Found")
            return None

    return None


In [12]:
def preload_translation_model():
    print("Loading Translation Model (Optimized)...")

    translation_model_path = TRANSLATION_MODEL.get(TRANSLATION_model_select)

    try:
        import torch
        from transformers import MarianMTModel, MarianTokenizer

        torch.set_num_threads(2)
        torch.set_grad_enabled(False)

        tokenizer = MarianTokenizer.from_pretrained(
            translation_model_path,
            local_files_only=True
        )

        model = MarianMTModel.from_pretrained(
            translation_model_path,
            local_files_only=True
        )

        model.eval()

        # Dynamic Quantization
        model = torch.quantization.quantize_dynamic(
            model,
            {torch.nn.Linear},
            dtype=torch.qint8
        )

        print("✔ MarianMT (Dynamic INT8 Quantized) Loaded")
        return tokenizer, model

    except Exception as e:
        print("Translation Load Error:", e)
        return None, None


In [13]:
import warnings
import multiprocessing as mp

if __name__ == "__main__":
    mp.set_start_method("spawn", force=True)

    print("\n=== Starting Offline Hindi Voice Assistant Preload ===\n")
    warnings.filterwarnings("ignore")

    asr_session = preload_asr_model()
    intent_model = preload_intent_model()
    tts_model = preload_tts_model()
    tokenizer,translator =preload_translation_model()
    print("All model checks completed.\n")
    print("Initialization complete.")
    print("System ready.\n")

    # Pipeline execution will start here (next stage)



=== Starting Offline Hindi Voice Assistant Preload ===

Loading ASR model (Optimized)...
ASR Load Error: No module named 'stt'
Loading Intent Model (Optimized)...
✔ Intent Model Loaded
Loading TTS Engine...
✔ eSpeak-NG Ready (System Level)
Loading Translation Model (Optimized)...
✔ MarianMT (Dynamic INT8 Quantized) Loaded
All model checks completed.

Initialization complete.
System ready.

